In [1]:
%load_ext autoreload
%autoreload 2

### imports and utilities

In [2]:
!pip install nltk scikit-learn matplotlib scipy -q

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [4]:
from collections import Counter
from utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.stats import norm
from scipy.stats import entropy as calculate_entropy

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Program Files\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


### load dataset

In [5]:
dataset = "newsgroup"

# total number of samples needed
randomize = False

# retrieve dataset
categories = ['rec.autos', 'talk.politics.mideast', 'alt.atheism', 'sci.space']

all_docs = fetch_20newsgroups(subset='train', shuffle=randomize, remove=('headers', 'footers', 'quotes'), categories=categories)
all_docs, old_labels, categories = all_docs.data, all_docs.target, all_docs.target_names

In [6]:
# dataset = "bbc"

# data = pd.read_csv('bbcsport.csv')

# all_docs = data["text"].to_list()
# old_labels = data["topic"].to_list()
# categories = classes = np.unique(data["topic"]).tolist()

### clean dataset

In [48]:
datasize = 90
min_document_length = 160
max_document_length = 256


index = -1
docs, labels, label_indices = [], [], []

sizes = [0]*len(categories)

with tqdm(total=len(categories)*datasize) as pbar:
    while sum(sizes) != len(categories)*datasize:
        index += 1
        label_index = old_labels[index]
            
        if sizes[label_index] == datasize:
            continue
        
        doc = all_docs[index]
        status, doc, word_count = clean_doc(doc, True)
        
        if not status:
            continue
            
        if min_document_length is not None and len(doc) < min_document_length:
            continue
            
        if max_document_length is not None and len(doc) > max_document_length:
            continue
        
        label_indices.append(label_index)
        labels.append(categories[label_index])
        
        docs.append(doc)
        sizes[label_index] += 1
        pbar.update(1)

labels = np.array(labels)
label_indices = np.array(label_indices)

KeyboardInterrupt: 

In [ ]:
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

In [ ]:
print(sizes)
assert min(sizes) == max(sizes) == datasize

### Split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size =.3)

In [ ]:
print(f"there are {len(docs)} total docs, {len(y_train)} train and {len(y_test)} test")

### Initialize Vectorizer

In [ ]:
# initialize the count vectorizer
vectorizer = CountVectorizer()

# fit it to dataset
vectorizer.fit(x_train)

vocabulary = np.array(vectorizer.get_feature_names())
print("word_count is", len(vocabulary))

### Prepare Datatset

In [ ]:
# create doc count vectors
train_doc_vectors = vectorizer.transform(x_train).toarray()
test_doc_vectors = vectorizer.transform(x_test).toarray()

wdf_train = pd.DataFrame(train_doc_vectors, columns=vocabulary)
wdf_test = pd.DataFrame(test_doc_vectors, columns=vocabulary)

## Word Word Probability Distr

In [ ]:
wdt_train = wdf_train.copy()
wdt_test = wdf_test.copy()

wdt_test["__labels__"] = y_test
wdt_train["__labels__"] = y_train

word_doc_count = wdf_train.sum(0)
word_word_pr_distr = pd.DataFrame(data=0.0, columns=vocabulary, index=vocabulary)

for word in tqdm(vocabulary):
    pxy = wdf_train[wdf_train[word] > 0].sum(0) / word_doc_count[word]
#     pxy = (wdf_train.T * wdf_train[word]).sum(1)/ word_doc_count[word]
    word_word_pr_distr[word] = pxy * (word_doc_count[word] / word_doc_count)

word_word_pr_distr /= word_word_pr_distr.max().max()
print(f"word_word_pr_distr shape = {word_word_pr_distr.shape}")

In [ ]:
word_word_pr_distr.head()

In [ ]:
word = "space"

topwords_df = word_word_pr_distr.loc[word].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Topwords: Word - {word}\n{'='*50}")
print(topwords_df)

### Modify Word Word Distr with entopy

In [ ]:
word_word_pr_entropy = calculate_entropy(word_word_pr_distr, axis=1)
word_word_pr_entropy_softmax = softmax(word_word_pr_entropy.max() - word_word_pr_entropy)

In [ ]:
word_word_pr_distr_prime = word_word_pr_distr.copy()

for word in tqdm(word_word_pr_distr.columns):
    word_word_pr_distr_prime.loc[word] *= word_word_pr_entropy_softmax

word_word_pr_distr_prime /= word_word_pr_distr_prime.max().max()

In [ ]:
word = "space"

topwords_prime_df = word_word_pr_distr_prime.loc[word].sort_values(ascending=False).head(10)
topwords_prime = topwords_prime_df.index.tolist()

print(f"Topwords Prime: Word - {word}\n{'='*50}")
print(topwords_prime_df)

### Compare Word Word Distr with Modified Word Word Distr

In [ ]:
base_word = "space"
li = {
    "against self": [base_word],
    "against topwords" : topwords, 
    "against topwords prime": topwords_prime
}

for title, other_words in li.items():
    print(f'{" ".join([base_word, title]):^33s}')
    print("===============================")
    print("word            | wwpd  | wwpd_prime")
    print("-------------------------------")
    
    for other_word in other_words:
        r1 = word_word_pr_distr.loc[base_word][other_word] / word_word_pr_distr.loc[base_word].max()
        r2 = word_word_pr_distr_prime.loc[base_word][other_word] / word_word_pr_distr_prime.loc[base_word].max()

        print(f"{other_word:15s} | {r1:5.2f} | {r2:5.2f}")
    print()

### Document Topwords

In [ ]:
doc_index = 0
print(f"Topic: {y_train[doc_index]}\n{'='*50}\n{x_train[doc_index][:512]}")

In [ ]:
wdf_train_original = wdf_train / wdf_train.max().max()
wdf_test_original = wdf_test / wdf_test.max().max()
 
topwords_df = wdf_train_original.loc[doc_index].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_df)

### Modify Word Doc Distr with Word Word Distr

In [ ]:
wdf_train_x = 0 * wdf_train_original
wdf_test_x = 0 * wdf_test_original
    
for wx, wo in [(wdf_train_x, wdf_train_original), (wdf_test_x, wdf_test_original)]:
    for doc_index in tqdm(range(len(wo))):
        denom = 0

        indices = (wo.loc[doc_index] > 0)
        xv = wo.loc[doc_index][indices]

        for index, (wordx, word_pr) in enumerate(xv.iteritems()):
            denom += word_pr
            wx.loc[doc_index] += word_pr * word_word_pr_distr.loc[wordx]

        wx.loc[doc_index] /= denom
    wx /= wx.max().max()

In [ ]:
topwords_df = wdf_train_x.loc[doc_index].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_df)

### Modify Word Doc Distr with Word Word Distr Prime

In [ ]:
wdf_train_prime = 0 * wdf_train_original
wdf_test_prime = 0 * wdf_test_original
    
for wx, wo in [(wdf_train_prime, wdf_train_original), (wdf_test_prime, wdf_test_original)]:
    for doc_index in tqdm(range(len(wo))):
        denom = 0

        indices = (wo.loc[doc_index] > 0)
        xv = wo.loc[doc_index][indices]

        for index, (wordx, word_pr) in enumerate(xv.iteritems()):
            denom += word_pr
            wx.loc[doc_index] += word_pr * word_word_pr_distr_prime.loc[wordx]

        wx.loc[doc_index] /= denom
    wx /= wx.max().max()

In [ ]:
doc_index = 0

topwords_prime_df = wdf_train_prime.loc[doc_index].sort_values(ascending=False).head(10)
topwords_prime = topwords_prime_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_prime_df)

In [ ]:
li = {
    "against topwords" : topwords, 
    "against topwords prime": topwords_prime
}

for title, other_words in li.items():
    print(f'{" ".join([f"doc {doc_index}", title]):^40s}')
    print("===============================")
    print("word            | wdpdo | wdpdx | wdpd_prime")
    print("-------------------------------")
    
    for other_word in other_words:
        r1 = wdf_train_original.loc[doc_index][other_word] / wdf_train_original.loc[doc_index].max()
        r2 = wdf_train_x.loc[doc_index][other_word] / wdf_train_x.loc[doc_index].max()
        r3 = wdf_train_prime.loc[doc_index][other_word] / wdf_train_prime.loc[doc_index].max()

        print(f"{other_word:15s} | {r1:5.2f} | {r2:5.2f} | {r3:5.2f}")
    print()

In [ ]:
wxs = [
    ("wdf_train", wdf_train),
    ("wdf_train_original", wdf_train_original),
    ("wdf_train_x", wdf_train_x),
    ("wdf_train_prime", wdf_train_prime)
]

for i in range(2, 7):
    print(f"using {i} classes")
    
    for title, wx in wxs:
        kmeans = KMeans(n_clusters=i, random_state=0).fit(wx)
        
        print(title, kmeans.inertia_)
        print(Counter(y_train[kmeans.labels_ == 0]))
        print(Counter(y_train[kmeans.labels_ == 1]))
        print(Counter(y_train[kmeans.labels_ == 2]))
        print(Counter(y_train[kmeans.labels_ == 3]))
        print()